### Script to create the datastet used by the app

In [1]:
import pandas as pd
import numpy as np
# %pip install kagglehub
import kagglehub

/opt/anaconda3/envs/jbi010/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Download latest emoji dataset version
# path = kagglehub.dataset_download("subinium/emojiimage-dataset")
# print("Path to dataset files:", path)

In [43]:
countries = pd.read_csv("countriesWorld.csv")

In [56]:
countries.head(5)

,Country,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
0,Afghanistan,ASIA (EX. NEAR EAST),31056997,647500,"48,0","0,00","23,06","163,07",700.0,"36,0","3,2","12,13","0,22","87,65",1,"46,6","20,34","0,38","0,24","0,38"
1,Albania,EASTERN EUROPE,3581655,28748,"124,6","1,26","-4,93","21,52",4500.0,"86,5","71,2","21,09","4,42","74,49",3,"15,11","5,22","0,232","0,188","0,579"
2,Algeria,NORTHERN AFRICA,32930091,2381740,"13,8","0,04","-0,39",31,6000.0,"70,0","78,1","3,22","0,25","96,53",1,"17,14","4,61","0,101","0,6","0,298"
3,American Samoa,OCEANIA,57794,199,"290,4","58,29","-20,71","9,27",8000.0,"97,0","259,5",10,15,75,2,"22,46","3,27",NaN,NaN,NaN
4,Andorra,WESTERN EUROPE,71201,468,"152,1","0,00","6,6","4,05",19000.0,"100,0","497,2","2,22",0,"97,78",3,"8,71","6,25",NaN,NaN,NaN


In [49]:
countries.Country = countries.Country.apply(lambda x: x.strip(" "))

In [53]:
emojis = pd.read_csv("2/full_emoji.csv", usecols=["emoji", "unicode", "name", "Apple"])
flags = emojis[emojis["name"].str.contains("flag:") == True]
flags["countryName"] = flags["name"].str.split(": ").str[1]
flags.rename(
    columns={"name": "emojiName"}, 
    inplace=True
)

/var/folders/6r/fy40h3sj5sz87ctgpzyw1lgm0000gn/T/ipykernel_66018/1294549689.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flags["countryName"] = flags["name"].str.split(": ").str[1]
/opt/anaconda3/envs/jbi010/lib/python3.9/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [55]:
flags.head(5)

,emoji,unicode,emojiName,Apple,countryName
1555,🇦🇨,U+1F1E6 U+1F1E8,flag: Ascension Island,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Ascension Island
1556,🇦🇩,U+1F1E6 U+1F1E9,flag: Andorra,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Andorra
1557,🇦🇪,U+1F1E6 U+1F1EA,flag: United Arab Emirates,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",United Arab Emirates
1558,🇦🇫,U+1F1E6 U+1F1EB,flag: Afghanistan,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Afghanistan
1559,🇦🇬,U+1F1E6 U+1F1EC,flag: Antigua & Barbuda,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Antigua & Barbuda


In [51]:
countries[countries["Country"] == "United States"]

,Country,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
214,United States,NORTHERN AMERICA,298444215,9631420,"31,0","0,21","3,41","6,5",37800.0,"97,0","898,0","19,13","0,22","80,65",3,"14,14","8,26","0,01","0,204","0,787"


In [52]:
flags[flags["countryName"] == "United States"]

,emoji,unicode,emojiName,Apple,countryName,countryName_clean
1795,🇺🇸,U+1F1FA U+1F1F8,flag: United States,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",United States,united states


In [73]:
from rapidfuzz.process import extractOne
import pandas as pd

# Step 1: Clean and normalize
flags['countryName_clean'] = flags['countryName'].str.strip().str.lower()
countries['Country_clean'] = countries['Country'].str.strip().str.lower()

# Step 2: Fuzzy match each emoji country name to a country
matches = []
for emoji_country in flags['countryName_clean']:
    match_name, score, _ = extractOne(emoji_country, countries['Country_clean'].tolist())
    matches.append((emoji_country, match_name, score))

# Step 3: Build match DataFrame
match_df = pd.DataFrame(matches, columns=["countryName_clean", "matched_country_clean", "similarity_score"])

# Step 4: Merge fuzzy match results back into flags
flags_matched = flags.merge(match_df, on="countryName_clean", how="left")

# Step 5: Merge matched flags with countries metadata
final_df = flags_matched.merge(
    countries,
    left_on="matched_country_clean",
    right_on="Country_clean",
    how="left"
)


/var/folders/6r/fy40h3sj5sz87ctgpzyw1lgm0000gn/T/ipykernel_66018/1508252774.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flags['countryName_clean'] = flags['countryName'].str.strip().str.lower()


In [78]:
# Sort by similarity descending so the highest comes first
final_df_sorted = final_df.sort_values(by='similarity_score', ascending=False)

# Drop duplicates keeping only the best match for each flag entry
final_df_unique = final_df_sorted.drop_duplicates(subset='countryName', keep='first')